# Model selection

In [1]:
# results path
import os
results_path = 'Results/ModelSelection/'
if not os.path.exists(results_path):
    os.makedirs(results_path)

In [304]:
"""Functions"""
import GCA
import random
import numpy as np
import numpy.matlib
from numpy import linalg as LA
import math as m
from numpy.linalg import det,inv,norm
from scipy import linalg, stats, sparse
from scipy.sparse import tril
from scipy.special import multigammaln
from scipy.stats import entropy
from scipy.stats import multivariate_normal
from sklearn.utils.extmath import fast_logdet
from sklearn.datasets import make_sparse_spd_matrix
from sklearn.covariance import GraphicalLassoCV, GraphicalLasso
import matplotlib.pyplot as plt
import networkx as nx
import time
import pickle
import copy
from cvxopt import solvers, matrix, spmatrix, log, mul, blas, lapack, amd, cholmod
import itertools


# #############################################################################
def covsel1(Y , K):
    """
    Returns the solution of
 
        minimize    -log det K + tr(KY)
        subject to  K_ij = 0  if (i,j) not in zip(I, J).
    Y is a symmetric sparse matrix with nonzero diagonal elements.
    I = Y.I,  J = Y.J.
    Here we use estimation of precision matrix from the previous step K as initialization point
    """

    cholmod.options['supernodal'] = 2

    I, J = Y.I, Y.J
    n, m = Y.size[0], len(I) 
    # non-zero positions for one-argument indexing 
    N = I + J*n         
    # position of diagonal elements
    D = [ k for k in range(m) if I[k]==J[k] ]  
    
    # Kn is used in the line search
    Kn = spmatrix(0.0, I, J)

    # symbolic factorization of K 
    F = cholmod.symbolic(K)

    # Kinv will be the inverse of K
    Kinv = matrix(0.0, (n,n))

    for iters in range(100):
        # numeric factorization of K
        cholmod.numeric(K, F)
        d = cholmod.diag(F)

        # compute Kinv by solving K*X = I 
        Kinv[:] = 0.0
        Kinv[::n+1] = 1.0
        cholmod.solve(F, Kinv)
        
        # solve Newton system
        grad = 2 * (Y.V - Kinv[N])
        hess = 2 * ( mul(Kinv[I,J], Kinv[J,I]) + 
               mul(Kinv[I,I], Kinv[J,J]) )
        v = -grad
        lapack.posv(hess,v) 
                                                  
        # stopping criterion
        sqntdecr = -blas.dot(grad,v) 
        #print("Newton decrement squared:%- 7.5e" %sqntdecr)
        if (sqntdecr < 1e-12):
            #print("number of iterations: %d" %(iters+1))
            break

        # line search
        dx = +v
        dx[D] *= 2      
        f = -2.0*sum(log(d))      # f = -log det K
        s = 1
        for lsiter in range(50):
            Kn.V = K.V + s*dx
            try: 
                cholmod.numeric(Kn, F)
            except ArithmeticError: 
                s *= 0.5
            else:
                d = cholmod.diag(F)
                fn = -2.0 * sum(log(d)) + 2*s*blas.dot(v,Y.V)
                if (fn < f - 0.01*s*sqntdecr): break
                else: s *= 0.5

        K.V = Kn.V
    C = np.zeros([K.size[0],K.size[0]])
    for i,j,v in zip(K.I,K.J,K.V):
        C[i,j] = v
        C[j,i] = v
    return inv(C)

def scipy_sparse_to_spmatrix(A):
    coo = A.tocoo()
    SP = spmatrix(coo.data.tolist(), coo.row.tolist(), coo.col.tolist(), size=A.shape)
    return SP

def Codelength_UniversalCoder(n,GG):
    cl= np.zeros(8)
    GG.remove_edges_from(nx.selfloop_edges(GG))
    pg = GCA.CS12s()
    pg.parse(n, GG, update=True)
    cons = n - 0.5 * np.log2(n)
    cl[0] = pg.codelength_bin() 
    cl[1] = pg.codelength_degree() + cons 
    cl[2] = pg.codelength_class1(ctype='tri') 
    cl[3] = pg.codelength_class2(ctype='tri') + cons 
    cl[4] = pg.codelength_class1(ctype='com') 
    cl[5] = pg.codelength_class2(ctype='com') + cons 
    cl[6] = pg.codelength_class1(ctype='4nod') 
    cl[7] = pg.codelength_class2(ctype='4nod')+ cons
    return cl*m.log(2)

def Codelength_data2(n,GG,X, B=10):
    """calculate codelength of data based on estimated covaraince. Here we estimated covariance
    from each step to initilize the next step A2. Also we encode using block coding with B blocks
    """
    sigma = []
    A = nx.to_scipy_sparse_matrix(GG)
    A = tril(A)
    A_hat= scipy_sparse_to_spmatrix(A)
    A_adj = nx.adjacency_matrix(GG).todense()
    cl_est = 0
    start = int(0.25*X.shape[1]) # where to start coding with estimated covariance matrix
    # default distribution for first few samples
    for j in range(start):
        cl_est += 0.5*n*m.log(2*m.pi) + 0.5*np.dot(X[j,:],X[j,:])
    
    # estimated covariance matrix for the rest of samples
    A2 = spmatrix(0.0, A_hat.I , A_hat.J) # starting point: symmetric identity with nonzero pattern I,J
    A2[::n+1] = 1.0
    prec_est = np.ones([n, n])
    M = np.ceil((X.shape[0]-start)/B).astype(int)
    for j in range(B):
        s1 = min(start+j*M, X.shape[0])
        s2 = min(X.shape[0]-s1, M)
        sigma_emp = np.matmul(X[:s1,:].transpose(),X[:s1,:])/s1
        A1 = spmatrix([sigma_emp[r,p] for r,p in zip(A_hat.I,A_hat.J)], A_hat.I , A_hat.J)
        sigma_est = covsel1(A1, A2)
        prec_est = inv(sigma_est)
        A2 = spmatrix([prec_est[r,p] for r,p in zip(A_hat.I,A_hat.J)], A_hat.I , A_hat.J)
        cl_est += s2*(0.5*m.log(det(sigma_est)) + 0.5*n*m.log(2*m.pi))
        for k in range(s2):
            cl_est += 0.5*np.dot(X[s1+k,:]@inv(sigma_est),X[s1+k,:])
    return cl_est


def Codelength_AllStructures2(X_, prec_mats):
    """By initializing covariance matrix for Dempster method and also block coding"""
    CL_data = []
    CL_graph = []
    for j in range(len(prec_mats)):
        G_ = nx.Graph(prec_mats[j])
        temp = Codelength_UniversalCoder(prec_mats[j].shape[0],G_)
        CL_graph.append(temp)
        
        G_ = nx.Graph(prec_mats[j])
        temp2 = Codelength_data2(prec_mats[j].shape[0],G_, X_, B=10)
        CL_data.append(temp2)
    
    return CL_data, CL_graph

def prec_generation (graph_type, dim, alpha=0.02):
    # graph_type: circular, AR4, ER
    # alpha: prob. of having edge in ER graph
    prec = np.zeros([dim,dim])
    if graph_type == 'cycle':
        for i in range(dim):
            for j in range(dim):
                if abs(i-j) == 1: prec[i,j] = 0.45
        prec[0,dim-1]= prec[dim-1,0]=0.45
        prec += np.eye(dim)
    
    if graph_type == 'AR1':
        for i in range(dim):
            for j in range(dim):
                if abs(i-j) == 1: prec[i,j] = 0.45
        prec += np.eye(dim)
        
    if graph_type == 'ER':
        for i in range(dim-1):
            for j in range(i+1,dim):
                if random.random() <= alpha: prec[i,j] = random.uniform(0.4,0.8)
        prec += prec.transpose()
        w, v = np.linalg.eig(prec)
        prec += (abs(min(w)) + 0.05)*np.eye(dim)
        d = np.sqrt(np.diag(prec))
        prec /= d 
        prec /= d[:, np.newaxis]
    
    if graph_type == 'Hub':
        tr = 0.01
        n_hubs = 2
        for i in range(dim-1):
            for j in range(i+1,dim):
                if random.random() <= tr:
                    prec[i,j] = 1
        hubs = random.sample(range(dim), n_hubs)
        for i in hubs:
            for j in range(dim):
                if i!=j:
                    if random.random() <= 0.7: prec[i,j] = 1
                    else: prec[i,j] = 0
        for i in range(dim-1):
            for j in range(i+1,dim):
                if i!=j and prec[i,j]==1:
                    if (random.random() <= 0.5): prec[i,j] = random.uniform(-0.75,-0.25)
                    else: prec[i,j] = random.uniform(0.25,0.75)

        prec = 0.5*(prec+prec.transpose())
        w, v = np.linalg.eig(prec)
        prec += (0.1 + abs(min(w)))*np.eye(dim)
        d = np.sqrt(np.diag(prec))
        prec /= d
        prec /= d[:, np.newaxis]
    
    if graph_type == 'BA':
        m = 2 # Number of edges to attach from a new node
        p = 0.5 # probability of adding a triangle after adding a random edge in power-law cluster model
        G = nx.barabasi_albert_graph(dim, m)
        #G = nx.powerlaw_cluster_graph(dim, m, p)
        prec_temp = nx.to_numpy_array(G).astype('int')
        for i in range(dim-1):
            for j in range(i+1,dim):
                if i!=j and prec_temp[i,j]==1:
                    if (random.random() <= 0.5): prec[i,j] = random.uniform(-0.75,-0.25)
                    else: prec[i,j] = random.uniform(0.25,0.75)

        prec = 0.5*(prec+prec.transpose())
        w, v = np.linalg.eig(prec)
        prec += (0.1 + abs(min(w)))*np.eye(dim)
        d = np.sqrt(np.diag(prec))
        prec /= d
        prec /= d[:, np.newaxis]
    
    if graph_type == 'WS':
        pr = 0.25  # probability of rewiring each edge
        k = 4 # Each node is joined with its k nearest neighbors
        G = nx.watts_strogatz_graph(dim, k, pr)
        prec_temp = nx.to_numpy_array(G).astype('int')
        for i in range(dim-1):
            for j in range(i+1,dim):
                if i!=j and prec_temp[i,j]==1:
                    if (random.random() <= 0.5): prec[i,j] = random.uniform(-0.75,-0.25)
                    else: prec[i,j] = random.uniform(0.25,0.75)

        prec = 0.5*(prec+prec.transpose())
        w, v = np.linalg.eig(prec)
        prec += (0.1 + abs(min(w)))*np.eye(dim)
        d = np.sqrt(np.diag(prec))
        prec /= d
        prec /= d[:, np.newaxis]
    
    return prec

def generate_sample(prec, n_samp, n_feat):
    prng = np.random.RandomState()
    cov = linalg.inv(prec)
    d = np.sqrt(np.diag(cov))
    cov /= d
    cov /= d[:, np.newaxis]
    x = prng.multivariate_normal(np.zeros(n_feat), cov, size=n_samp)
    x -= x.mean(axis=0)
    x /= x.std(axis=0)
    return x


In [305]:
"""Functions for L1 solvers and model selection techniques"""
import numpy as np
from sklearn.model_selection import KFold
from sklearn.covariance import log_likelihood
from sklearn.utils.extmath import fast_logdet

try:
    import rpy2
    from rpy2.robjects.packages import importr
    import rpy2.robjects.numpy2ri as swap

    R = rpy2.robjects.r
    importr('glasso')
    importr('BigQuic')
    importr('dpglasso')
except:
    msg = "Either R, rpy2, or BigQuic are not installed"
    raise ImportError(msg)


def glasso(data, alpha=1.):
    """Python front-end for GLASSO solver."""
    data = np.copy(data)
    cov_emp = np.dot(data.T, data) / data.shape[0]
    S = swap.numpy2rpy(cov_emp)  # Load data into R space
    # Construct and run the program in R
    program_string = 'f <- function(r) {out = glasso(r, rho=%a); ' % alpha
    program_string += 'as(out$wi, "matrix")}'
    f = R(program_string)
    prec = np.array(f(S))
    return prec

def bigquic(data, alpha=1.):
    """Python front-end for BigQuic solver."""
    data = np.copy(data)
    rdata = swap.numpy2rpy(data)  # Load data into R space

    # Construct and run the program in R
    program_string = 'f <- function(r) {out = BigQuic(X=r, lambda=%a, use_ram=TRUE); ' % alpha
    program_string += 'as(out$precision_matrices[[1]], "matrix")}'
    f = R(program_string)
    prec = np.array(f(rdata))
    return prec

def dpglasso(data, alpha=1.):
    """Python front-end for dpglasso solver."""
    data = np.copy(data)
    cov_emp = np.dot(data.T, data) / data.shape[0]
    S = swap.numpy2rpy(cov_emp)  # Load data into R space
    # Construct and run the program in R
    #program_string = 'f <- function(r) {out = dpglasso(r, rho=%a); ' % alpha
    program_string = 'f <- function(r) {out = dpglasso(r, rho=%a, outer.tol=%a); ' % (alpha, 10e-3) # set tolerance
    program_string += 'as(out$X, "matrix")}'
    f = R(program_string)
    prec = np.array(f(S))
    return prec

def find_range(emp_cov):
    """Find range for lambda"""
    A = np.copy(emp_cov)
    A.flat[::A.shape[0]+1] = 0
    alpha_1 = np.max(np.abs(A))
    alpha_0 = 1e-1 * alpha_1
    n_alphas = 50
    range_ = np.logspace(np.log10(alpha_0), np.log10(alpha_1) , n_alphas)
    return range_

def solver_path(data, lamb, solver):
    """ Solve L1 log-likelihood for all lambdas and return precision matrices """
    num_nodes = data.shape[1]
    num_sample = data.shape[0]
    prec_matrix = [np.zeros([num_nodes, num_nodes]) for j in range(len(lamb))]
    
    for j in range(len(lamb)):
        if solver == 'glasso': prec_ = glasso(data, lamb[j])
        elif solver == 'dpglasso': prec_ = dpglasso(data, lamb[j])
        elif solver == 'bigquic': prec_ = bigquic(data, lamb[j])
        prec_matrix[j] += prec_
    return prec_matrix

def solver_single(data, lamb, solver):
    """ Solve L1 log-likelihood for single lambda and return precision matrix """
    if solver == 'glasso': prec_ = glasso(data, lamb)
    elif solver == 'dpglasso': prec_ = dpglasso(data, lamb)
    elif solver == 'bigquic': prec_ = bigquic(data, lamb)
    return prec_

def stars_metric(data, lamb, solver, beta=0.05, N=20):
    """
    Implementation of the StARS (Stability Approach to Regularization Selection) algorithm [Liu et al., 2010]
       
    lamb: vector of lambda by increasing values (for example lamb = np.logspace(-1,0,10))
    data: data
    beta: cut point value
    N: number of bootstraps
    """
    num_nodes = data.shape[1]
    num_sample = data.shape[0]
    if num_sample > 144:
        num_subsample = int(10*np.sqrt(num_sample))
    else:
        num_subsample = int(0.8*num_sample)
    Adj_matrix = [np.zeros([num_nodes, num_nodes]) for j in range(len(lamb))]
    
    for ind_boot in range(N):
        # we consider 10*sqrt(num_sample) as the size of subsample set
        boot_index = np.random.choice(num_sample, num_subsample, replace=False) # Random subsample
        # solve sover for each value of lambda
        for j in range(len(lamb)):
            if solver == 'glasso': prec_ = glasso(data[boot_index, :], lamb[j])
            elif solver == 'dpglasso': prec_ = dpglasso(data[boot_index, :], lamb[j])
            elif solver == 'bigquic': prec_ = bigquic(data[boot_index, :], lamb[j])
            Adj_matrix[j] += 1*(prec_ != 0) # for each edge, add 1 if it is present in the network

    # Compute the total instability for each lambda
    v = np.zeros(len(lamb))
    iu = np.triu_indices(num_nodes, 1)
    for j in range(len(lamb)):
        Adj_matrix[j] = Adj_matrix[j] / N
        D = 2*Adj_matrix[j]*(1-Adj_matrix[j]) # instability of the edges across subsamples
        v[j] = np.mean(D[iu]) # total instability
    
    # Determine the optimal lambda
    vcum = [np.max(v[rho:]) for rho in range(len(lamb))]
    lambda_opt = np.min(lamb[np.array(vcum) <= beta])
    # solve the estimator with lambda_opt and return the precision matrix
    return lambda_opt

def cv_metric(x, alphas, solver, k=5):
    """Choose sparsity hyper-parameter using k-fold cross validation,
    The documentation recommends NOT to use alpha < 0.4 for performance reasons..."""
    kf = KFold(n_splits=k)
    cv_dict = {}
    for alpha in alphas:
        for x_train, x_test in kf.split(x):
            if solver == 'glasso': prec_ = glasso(x[x_train], alpha)
            elif solver == 'dpglasso': prec_ = dpglasso(x[x_train], alpha)
            elif solver == 'bigquic': prec_ = bigquic(x[x_train], alpha)
            emp_cov = np.dot(x[x_test].T, x[x_test]) / x[x_test].shape[0]
            score = log_likelihood(emp_cov, prec_)
            cv_dict[alpha] = cv_dict.get(alpha, []) + [score]
    best_alpha = sorted([(-np.mean(v), k) for k, v in cv_dict.items()])[0][1]
    return best_alpha

def ebic_metric(data, prec_mats, gamma=0):
    """
    Extended Bayesian Information Criteria for model selection.
    data : 2D ndarray (n_samples, n_features)
    precision : 2D ndarray (n_features, n_features)
        The precision matrix of the model to be tested
    gamma : (float) \in (0, 1)
        Choice of gamma=0 leads to classical BIC
        Positive gamma leads to stronger penalization of large graphs.
    Returns
    -------
    ebic score (float).  Caller should minimized this score.
    """
    n_samples = data.shape[0]
    n_features = data.shape[1]
    covariance = np.dot(data.T, data) / n_samples
    score = np.zeros(len(prec_mats))
    for i in range(len(prec_mats)):
        l_theta = 0
        l_theta = -np.sum(covariance * prec_mats[i]) + fast_logdet(prec_mats[i])
        l_theta *= n_features / 2.

        # if something goes wrong with fast_logdet, return large value
        if np.isinf(l_theta) or np.isnan(l_theta):
            l_theta = 1e10
        mask = np.abs(prec_mats[i].flat) > np.finfo(prec_mats[i].dtype).eps
        precision_nnz = (np.sum(mask) - n_features) / 2.0  # lower off diagonal tri
        score[i] = -2.0 * l_theta + precision_nnz * np.log(n_samples) + 4.0 * precision_nnz * np.log(n_features) * gamma
    
    idx = np.argmin(score)
    return idx

In [ ]:
"""Select parameteres"""
size = 100 # size of graph
iters = 50 # number of trials
n_features = size # number of variables
n_samples = 2*size # number of samples
# select graph strcuture
graph_type = ['cycle', 'AR1', 'ER', 'Hub', 'BA', 'WS']
graph = graph_type[-2]
# select solver
solvers = ['glasso', 'dpglasso', 'bigquic']
solver = solvers[1] # picke the solver

In [ ]:
"""Main function"""
data = []
precision = []
t0 = time.time()
##########################################################33
# data generation
for t in range(iters):      
    prng = np.random.RandomState(t)
    prec = prec_generation (graph, n_features)
    #prec = prec_generation (graph, n_features, 2/n_features)
    cov = linalg.inv(prec)
    d = np.sqrt(np.diag(cov))
    cov /= d
    cov /= d[:, np.newaxis]
    prec *= d
    prec *= d[:, np.newaxis]
    X = prng.multivariate_normal(np.zeros(n_features), cov, size=n_samples)
    X -= X.mean(axis=0)
    X /= X.std(axis=0)
    data.append(X)
    precision.append(prec)
##############################################################################
# estimate covariance matrix
EBIC = []
BIC = []
CV = []
STARS = []
CODING = []
for t in range(iters):
    if(t%5 == 0): print("iter:", t)
    X = np.copy(data[t])
    emp_cov = np.dot(X.T, X) / n_samples
    lambdaRange = find_range(emp_cov) # for n_samples > n_features
    #lambdaRange = np.logspace(-1, 0, 50) # for n_samples < n_features
    prec_path = solver_path(X, lambdaRange, solver)
    
    lambda_stars = stars_metric(X, lambdaRange, solver)
    prec_stars = solver_single(X, lambda_stars, solver)
    STARS.append(prec_stars)
    
    lambda_cv = cv_metric(X, lambdaRange, solver)
    prec_cv = solver_single(X, lambda_cv, solver)
    CV.append(prec_cv)
    
    bias = 15
    cl_d, cl_g = Codelength_AllStructures2(X, prec_path[bias:]) # coding wih one precision matrix
    #cl_d, cl_g = Codelength_AllStructures3(X, prec_path[bias:-10], 100) # coding with precision submatrices size as input
    coding_prec = []
    for d in range(8):
        coding_idx = np.argmin([i+j for i,j in zip(cl_d, [cl[d] for cl in cl_g])])
        coding_prec.append(prec_path[coding_idx+bias])
    CODING.append(coding_prec)
    
    ebic_idx = ebic_metric(X, prec_path, 0.5)
    EBIC.append(prec_path[ebic_idx])
    
    bic_idx = ebic_metric(X, prec_path)
    BIC.append(prec_path[bic_idx])
    
print("total time: ",time.time() - t0)

In [420]:
"""save input data"""
f = [data, precision, EBIC, BIC, CV, STARS, CODING]
with open(results_path+ graph + '_'+solver +'_feature_%s_sample_%s.text'%(n_features, n_samples),'wb') as fp:
    pickle.dump(f, fp)

In [423]:
def perf_metrics(true, est):
    metrics = []
    for i in range(len(true)):
        bool_true = true[i] != 0
        bool_est = est[i] != 0
        common_edge = np.sum(bool_true & bool_est)
        diag = bool_true.shape[0]
        precision = round((common_edge - diag) / (np.sum(bool_est) - diag),3)
        recall = round((common_edge - diag) / (np.sum(bool_true) - diag),3)
        F1 = round(2*precision*recall /(precision + recall),3)    
        metrics.append([precision, recall, F1])
    return metrics

CV_metrics = perf_metrics(precision, CV)
BIC_metrics = perf_metrics(precision, BIC)
EBIC_metrics = perf_metrics(precision, EBIC)
STARS_metrics = perf_metrics(precision, STARS)
CODING_metrics = []
for i in range(8):
    mats = [mat[i] for mat in CODING]
    CODING_metrics.append(perf_metrics(precision, mats))    

In [424]:
print('mean results for', solver + ', ' + graph)
print('cv:', np.round(np.nanmean(CV_metrics, 0),2))
print('stars:', np.round(np.nanmean(STARS_metrics, 0),2))
print('bic:', np.round(np.nanmean(BIC_metrics, 0),2))
print('ebic:', np.round(np.nanmean(EBIC_metrics, 0),2))
print('graph coding:')
print('iid:', np.round(np.nanmean(CODING_metrics[0], 0), 2))
print('class1_tri:', np.round(np.nanmean(CODING_metrics[2], 0), 2))
print('class1_com:', np.round(np.nanmean(CODING_metrics[4], 0), 2)) 
print('class1_4nod:', np.round(np.nanmean(CODING_metrics[6], 0), 2))
print('class2_degree:', np.round(np.nanmean(CODING_metrics[1], 0), 2))
print('class2_tri:', np.round(np.nanmean(CODING_metrics[3], 0), 2))
print('class2_com:', np.round(np.nanmean(CODING_metrics[5], 0), 2))
print('class2_4nod:', np.round(np.nanmean(CODING_metrics[7], 0), 2))

mean results for dpglasso, BA
cv: [0.18 0.82 0.3 ]
stars: [0.52 0.56 0.54]
bic: [0.76 0.32 0.44]
ebic: [0.94 0.09 0.17]
graph coding:
iid: [0.77 0.34 0.46]
class1_tri: [0.75 0.35 0.47]
class1_com: [0.74 0.36 0.48]
class1_4nod: [0.75 0.35 0.48]
class2_degree: [0.77 0.34 0.46]
class2_tri: [0.75 0.35 0.47]
class2_com: [0.75 0.35 0.47]
class2_4nod: [0.75 0.35 0.47]


In [425]:
print('std results for', solver + ', ' + graph)
print('cv:', np.round(np.nanstd(CV_metrics, 0), 2))
print('stars:', np.round(np.nanstd(STARS_metrics, 0), 2))
print('bic:', np.round(np.nanstd(BIC_metrics, 0), 2))
print('ebic:', np.round(np.nanstd(EBIC_metrics, 0), 2))
print('graph coding:')
print('iid:', np.round(np.nanstd(CODING_metrics[0], 0), 2))
print('class1_tri:', np.round(np.nanstd(CODING_metrics[2], 0), 2))
print('class1_com:', np.round(np.nanstd(CODING_metrics[4], 0), 2))
print('class1_4nod:', np.round(np.nanstd(CODING_metrics[6], 0), 2))
print('class2_degree:', np.round(np.nanstd(CODING_metrics[1], 0), 2))
print('class2_tri:', np.round(np.nanstd(CODING_metrics[3], 0), 2))
print('class2_com:', np.round(np.nanstd(CODING_metrics[5], 0), 2))
print('class2_4nod:', np.round(np.nanstd(CODING_metrics[7], 0), 2))

std results for dpglasso, BA
cv: [0.01 0.05 0.02]
stars: [0.04 0.07 0.05]
bic: [0.09 0.09 0.09]
ebic: [0.06 0.04 0.07]
graph coding:
iid: [0.08 0.08 0.09]
class1_tri: [0.08 0.07 0.08]
class1_com: [0.08 0.08 0.08]
class1_4nod: [0.08 0.08 0.08]
class2_degree: [0.08 0.08 0.09]
class2_tri: [0.08 0.08 0.08]
class2_com: [0.08 0.08 0.08]
class2_4nod: [0.08 0.08 0.08]
